In [7]:
import pandas as pd
import scipy as sc
import numpy as np
import sklearn
import pickle
import pathlib as Path
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sb
sb.set(font='IPAexGothic')

import multiprocessing
import itertools
import collections
import datetime
import gc

from tqdm._tqdm_notebook import tqdm

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

data_path = Path.Path("../data")
result_path = Path.Path("../result")

prefix = 'ana005'

# データ抽出
> ・全体＋試合データ（all_gms_df.pkl）

In [8]:
all_df = pd.read_pickle(data_path / 'ana004_all_df.pkl')

In [9]:
all_df

,id,B,O,S,b1,b2,b3,ballPositionLabel,ballX,ballY,batter,batterHand,battingType,dir,dist,flg_train,gameID,inning,isOuts,pitchType,pitcher,pitcherHand,speed,totalPitchingCount,y,topTeam,bottomTeam,batterTeam,pitcherTeam
0,0,0,0,0,False,False,False,内角低め,17.0,J,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,ストレート,今永 昇太,L,149km/h,1,0.0,広島,DeNA,広島,DeNA
1,1,1,0,0,False,False,False,内角低め,14.0,I,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,ストレート,今永 昇太,L,149km/h,2,1.0,広島,DeNA,広島,DeNA
2,2,1,0,1,False,False,False,外角高め,8.0,D,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,チェンジアップ,今永 昇太,L,137km/h,3,0.0,広島,DeNA,広島,DeNA
3,3,2,0,1,False,False,False,内角中心,21.0,G,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,スライダー,今永 昇太,L,138km/h,4,2.0,広島,DeNA,広島,DeNA
4,4,2,0,2,False,False,False,外角中心,7.0,F,ピレラ,R,G,S,38.3,1,20202173,1回表,False,チェンジアップ,今永 昇太,L,136km/h,5,4.0,広島,DeNA,広島,DeNA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54203,33803,0,0,1,False,False,False,NaN,NaN,NaN,坂口 智隆,L,NaN,NaN,NaN,0,20202023,5回表,NaN,NaN,床田 寛樹,L,NaN,2,NaN,ヤクルト,広島,ヤクルト,広島
54204,33804,0,0,0,False,False,False,NaN,NaN,NaN,メヒア,R,NaN,NaN,NaN,0,20202640,9回表,NaN,NaN,堀岡 隼人,R,NaN,1,NaN,広島,巨人,広島,巨人
54205,33805,0,0,0,True,False,False,NaN,NaN,NaN,鈴木 誠也,R,NaN,NaN,NaN,0,20202864,7回裏,NaN,NaN,ディプラン,R,NaN,1,NaN,巨人,広島,広島,巨人
54206,33806,3,1,1,False,True,False,NaN,NaN,NaN,周東 佑京,L,NaN,NaN,NaN,0,20202806,8回裏,NaN,NaN,田村 伊知郎,R,NaN,5,NaN,西武,ソフトバンク,ソフトバンク,西武


# データ加工
> ・ヒット以上有無（is_hit0）<br>
> ・シングルヒット（is_hit1) <br>
> ・2塁打（is_hit2）<br>
> ・3塁打（is_hit3）<br>
> ・ホームラン（is_hit4）<br>

In [11]:
# ヒット以上有無
all_df['is_hit0'] = (all_df['y'] >= 4)

# シングルヒット
all_df['is_hit1'] = (all_df['y'] == 4)
# 2塁打
all_df['is_hit2'] = (all_df['y'] == 5)
# 3塁打
all_df['is_hit3'] = (all_df['y'] == 6)
# ホームラン
all_df['is_hit4'] = (all_df['y'] == 7)

all_df.head(100)

,id,B,O,S,b1,b2,b3,ballPositionLabel,ballX,ballY,batter,batterHand,battingType,dir,dist,flg_train,gameID,inning,isOuts,pitchType,pitcher,pitcherHand,speed,totalPitchingCount,y,topTeam,bottomTeam,batterTeam,pitcherTeam,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4
0,0,0,0,0,False,False,False,内角低め,17.0,J,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,ストレート,今永 昇太,L,149km/h,1,0.0,広島,DeNA,広島,DeNA,False,False,False,False,False
1,1,1,0,0,False,False,False,内角低め,14.0,I,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,ストレート,今永 昇太,L,149km/h,2,1.0,広島,DeNA,広島,DeNA,False,False,False,False,False
2,2,1,0,1,False,False,False,外角高め,8.0,D,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,チェンジアップ,今永 昇太,L,137km/h,3,0.0,広島,DeNA,広島,DeNA,False,False,False,False,False
3,3,2,0,1,False,False,False,内角中心,21.0,G,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,スライダー,今永 昇太,L,138km/h,4,2.0,広島,DeNA,広島,DeNA,False,False,False,False,False
4,4,2,0,2,False,False,False,外角中心,7.0,F,ピレラ,R,G,S,38.3,1,20202173,1回表,False,チェンジアップ,今永 昇太,L,136km/h,5,4.0,広島,DeNA,広島,DeNA,True,True,False,False,False
5,5,0,0,0,True,False,False,真ん中低め,11.0,H,菊池 涼介,R,NaN,NaN,NaN,1,20202173,1回表,NaN,ストレート,今永 昇太,L,148km/h,1,2.0,広島,DeNA,広島,DeNA,False,False,False,False,False
6,6,0,0,1,True,False,False,外角高め,3.0,B,菊池 涼介,R,B,O,11.3,1,20202173,1回表,True,ストレート,今永 昇太,L,148km/h,2,3.0,広島,DeNA,広島,DeNA,False,False,False,False,False
7,7,0,1,0,True,False,False,内角中心,8.0,E,西川 龍馬,L,NaN,NaN,NaN,1,20202173,1回表,NaN,ストレート,今永 昇太,L,150km/h,1,1.0,広島,DeNA,広島,DeNA,False,False,False,False,False
8,8,0,1,1,True,False,False,ど真ん中,13.0,F,西川 龍馬,L,NaN,NaN,NaN,1,20202173,1回表,NaN,スライダー,今永 昇太,L,136km/h,2,2.0,広島,DeNA,広島,DeNA,False,False,False,False,False
9,9,0,1,2,True,False,False,内角中心,1.0,E,西川 龍馬,L,NaN,NaN,NaN,1,20202173,1回表,NaN,ストレート,今永 昇太,L,149km/h,3,0.0,広島,DeNA,広島,DeNA,False,False,False,False,False


# データ保存

In [12]:
all_df.to_pickle(data_path / ("%s_all_df.pkl" % prefix))